# Convergence

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from nodepy import rk
import cvxpy as cp


import scipy.linalg as linalg


#Diagonally Implicit methods:
BE = rk.loadRKM('BE').__num__()
SDIRK23 = rk.loadRKM('SDIRK23').__num__()
SDIRK34 = rk.loadRKM('SDIRK34').__num__()
SDIRK54 = rk.loadRKM('SDIRK54').__num__()
TR_BDF2 = rk.loadRKM('TR-BDF2').__num__()


be = rk.loadRKM('BE').__num__()

#Extrapolation method
ex2 = rk.extrap(2,'implicit euler').__num__()
ex3 = rk.extrap(3,'implicit euler').__num__()
ex4 = rk.extrap(4,'implicit euler').__num__()
ex5 = rk.extrap(5,'implicit euler').__num__()
ex6 = rk.extrap(6,'implicit euler').__num__()
ex8 = rk.extrap(8,'implicit euler').__num__()

bs5 = rk.loadRKM('BS5').__num__()
dp5 = rk.loadRKM('DP5').__num__()

from OrderCondition import *
from RKimple import *
import utils 


# Heat

In [ ]:
ex3 = rk.extrap(3,'implicit euler').__num__()
b_hat_1 = [np.array([0,0,0,1/3,1/3,1/3])]
b_hat_orig = [ex3.b]
ex3.b_hat = {3:b_hat_orig,2:b_hat_orig,1:b_hat_1}

In [ ]:
data = np.load('convergence_heat.npz')
for name in data.files:
    exec(name + '=data[\''+name+'\']')

In [ ]:
plt.loglog(dts,err_be,'C1',label = 'BE')
plt.loglog(dts,err_ex3,'C2',label = 'ex3')
plt.loglog(dts,err_ex3a,'C3',label = 'ex3a')
plt.loglog(dts,err_ex3c,'C4',label = 'ex3c')

plt.plot(dts[False==change_be],err_be[False==change_be],'oC1')
plt.plot(dts[False==change_ex3],err_ex3[False==change_ex3],'oC2')
plt.plot(dts[False==change_ex3a],err_ex3a[False==change_ex3a],'oC3')
plt.plot(dts[True==change_ex3a],err_ex3a[True==change_ex3a],'xC3')
plt.plot(dts[False==change_ex3c],err_ex3c[False==change_ex3c],'oC4')
plt.plot(dts[True==change_ex3c],err_ex3c[True==change_ex3c],'xC4')

plt.legend()
plt.grid()

plt.xlabel('$\Delta t$')
plt.ylabel('$e_{rel}$')

plt.savefig('conv_heat.pdf')

In [ ]:
np.min(dts[change_ex3a==True])

# Advectio Decay

In [ ]:
def f_upwind_advection_influx(t,u):
    dx = 1/len(u)
    du = np.zeros_like(u)
    du[1:] = -(u[1:]-u[:-1])
    du[0] = -u[0]+1 #boundary condition
    return du/(dx)

def f_adv_dec(t,u):
    a = 1 #advection constant
    K = 1 #decay constant K =10
    T_a = 0
    
    du = a *f_upwind_advection_influx(t,u)+K*(T_a-u)
    return du
    
def ref_adv_dec(N,t,dt=None,a=1,K=1):
    """
    Calculatest the exact solutio using a matrix exponential
    if no dt is given returns only the final value
    
    for more details on how the solution is constructed see seperate notebook
    """
    
    dx = 1/N

    A_adv = 1/dx * (-np.diag(np.ones(N))+np.diag(np.ones(N-1),-1))
    A_adv_dec = a*A_adv - K*np.diag(np.ones(N))
    L = np.zeros([N+1,N+1])
    L[1:,1:] = A_adv_dec #New right hand side
    L[1,0] = a/dx

    mu = np.zeros(N+1)#u_0 and initial Point
    mu[0] = 1
    

    if dt:
        t = np.arange(0,t+dt,dt)
        u = np.zeros([N,len(t)])
        u[:,0] = np.zeros(N)
        for i in range(1,len(t)):
            t_hat = t[i]
            u_hat = linalg.expm(t_hat*L)@mu
            u[:,i] = u_hat[1:]
        return t,u

    
    else:
        u_hat = linalg.expm(t*L)@mu
        return t,u_hat[1:]
    
  

N = 100
dx = 1/N
dt = 1.7*dx

dts = np.logspace(-2.2,np.log10(dt),num=20)
#dts = dts[dts<dt]
t_end = 0.5


#reference

reference = np.zeros([N,len(dts)])
for i in range(len(dts)):
    reference[:,i]= ref_adv_dec(N,t_end)[1]


u0 = np.zeros(N)
#u0[0]=1
problem_adv_dec = Problem(f=f_adv_dec,
                 u0=u0,
                 minval=0,
                 maxval=np.inf)

solver_fix = Solver(rkm = dp5,
               dt = dt,
               t_final = t_end,
               b_fixed=True,
               tol_neg=1e-11,
               tol_change = 5,
               p = [4,3,2,1],
               theta = [1],
               solver = cp.MOSEK,
               LP_opts = {})

solver_adp = Solver(rkm = dp5,
               dt = dt,
               t_final = t_end,
               b_fixed=False,
               tol_neg=1e-11,
               tol_change = np.inf,
               p = [4,3],
               theta = [1],
               solver = cp.MOSEK,
               LP_opts = {})

print('fixed')
sol_fix,err_fix,change_fix = utils.plot_convergence(problem_adv_dec,solver_fix,dts,reference,error='rel',step = -1,
                              Params=dict())

print('adapt')
sol_adp,err_adp,change_adp = utils.plot_convergence(problem_adv_dec,solver_adp,dts,reference,error='rel',step = -1,
                              Params=dict())

In [ ]:
np.savez('convergence_adde.npz', 
         dts=dts,
        sol_fix = sol_fix,err_fix = err_fix,change_fix = change_fix,
        sol_adp = sol_adp,err_adp=err_adp,change_adp=change_adp)

In [ ]:
data = np.load('convergence_adde.npz')
for name in data.files:
    exec(name + '=data[\''+name+'\']')

In [ ]:
plt.loglog(dts,err_fix,'C1',label = 'fix')
plt.loglog(dts,err_adp,'C2',label = 'adp')



plt.plot(dts[False==change_fix],err_fix[False==change_fix],'oC1')
plt.plot(dts[False==change_adp],err_adp[False==change_adp],'oC2')
plt.plot(dts[True==change_adp],err_adp[True==change_adp],'xC2')


plt.legend()
plt.grid()

plt.xlabel('$\Delta t$')
plt.ylabel('$e_{rel}$')

plt.savefig('conv_adde.pdf')

In [ ]:
plt.plot(sol_adp[-2])
plt.plot(reference[:,2])

In [ ]:
print(np.min(dts[change_adp == True]))
print(np.max(dts[change_adp == False]))
display(dts[-2])